<a href="https://colab.research.google.com/github/BroccoliWarrior/transformer-basic-knowledge/blob/main/tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

正向最大匹配法FMM

In [ ]:
def forward_max_matching(sentense, dictionary, max_len=6):
  result = []
  i = 0
  while i < len(sentense):
    matched = False
    for j in range(max_len, 0, -1):
      if i + j > len(sentense):
        continue
      word = sentense[i:i+j]
      if word in dictionary:
        result.append(word)
        i += j
        matched = True
        break
    if not matched:
      result.append(sentense[i])
      i += 1

  return result

dictionary = {'大模型', '学习', '模型', '我要', '要'}

sentense = "我要学习大模型"

print('/'.join(forward_max_matching(sentense, dictionary)))

我要/学习/大模型


逆向最大匹配法BMM

In [ ]:
def backward_max_matching(sentense, dictionary, max_len=6):
  result = []
  i = len(sentense)
  while i > 0:
    matched = False
    for j in range(max_len, 0, -1):
      if i - j < 0:
        continue
      word = sentense[i-j:i]
      if word in dictionary:
        result.insert(0, word)
        i -= j
        matched = True
        break
    if not matched:
      result.insert(0, sentense[i-1])
      i -= 1

  return result

dictionary = {'大模型', '学习', '模型', '我要', '要'}

sentense = "我要学习大模型"

print('/'.join(backward_max_matching(sentense, dictionary)))

我要/学习/大模型


根据**拆分粒度**的不同，Tokenization可以分为：
1.   词粒度-Word-based
2.   子词粒度-Subword-based
3.   字符粒度-Character-based

其中最常用的是**子词粒度**

reason1.
  词粒度：长尾效应和稀有词问题以及OOV
reason2.
  字符粒度：虽解决了OOV，但是处理效率低且丢失语义



下面介绍基于子词粒度的Tokenization方法：


**BPE：Byte-Pair Encoding**

  1.从 **字符级** 开始，把训练语料切分成最小单位（单个字符）

  2.然后逐步 合并出现频率最高的相邻符号对（bigram），生成新的子词单元

  3.直到 ***merge_rule*** 达到设定的 词表大小 (target_vocab_size) 或 合并次数 (merge operations)
  
  4.***利用merge_rule进行分词***

In [ ]:
from collections import Counter, defaultdict

def get_stats(vocab):
    """计算当前词表中所有相邻符号对的出现次数"""
    pairs = defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

def merge_vocab(pair, vocab):
    """合并最频繁的符号对"""
    bigram = ' '.join(pair)
    new_vocab = {}
    for word in vocab:
        new_word = word.replace(bigram, ''.join(pair))
        new_vocab[new_word] = vocab[word]
    return new_vocab

def bpe(vocab, num_merges):
    """执行 BPE 算法"""
    for i in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break
        # 找到出现次数最多的符号对
        best = max(pairs, key=pairs.get)
        vocab = merge_vocab(best, vocab)
        print(f'Merge {i+1}: {best}, vocab: {vocab}')
    return vocab

# 示例文本
corpus = ["low", "lower", "newest", "widest"]

# 初始化词表：每个词拆分为字符，并加入空格分隔符，统计词频
vocab = Counter()
for word in corpus:
    vocab[' '.join(list(word)) + ' </w>'] += 1

print("Initial vocab:", vocab)

# 执行 BPE
num_merges = 10
vocab = bpe(vocab, num_merges)
print("Final vocab:", vocab)


Initial vocab: Counter({'l o w </w>': 1, 'l o w e r </w>': 1, 'n e w e s t </w>': 1, 'w i d e s t </w>': 1})
Merge 1: ('l', 'o'), vocab: {'lo w </w>': 1, 'lo w e r </w>': 1, 'n e w e s t </w>': 1, 'w i d e s t </w>': 1}
Merge 2: ('lo', 'w'), vocab: {'low </w>': 1, 'low e r </w>': 1, 'n e w e s t </w>': 1, 'w i d e s t </w>': 1}
Merge 3: ('e', 's'), vocab: {'low </w>': 1, 'low e r </w>': 1, 'n e w es t </w>': 1, 'w i d es t </w>': 1}
Merge 4: ('es', 't'), vocab: {'low </w>': 1, 'low e r </w>': 1, 'n e w est </w>': 1, 'w i d est </w>': 1}
Merge 5: ('est', '</w>'), vocab: {'low </w>': 1, 'low e r </w>': 1, 'n e w est</w>': 1, 'w i d est</w>': 1}
Merge 6: ('low', '</w>'), vocab: {'low</w>': 1, 'low e r </w>': 1, 'n e w est</w>': 1, 'w i d est</w>': 1}
Merge 7: ('low', 'e'), vocab: {'low</w>': 1, 'lowe r </w>': 1, 'n e w est</w>': 1, 'w i d est</w>': 1}
Merge 8: ('lowe', 'r'), vocab: {'low</w>': 1, 'lower </w>': 1, 'n e w est</w>': 1, 'w i d est</w>': 1}
Merge 9: ('lower', '</w>'), vocab: {

Google提出了***SentencePiece***风格的BPE，加速了该算法

(1) 只维护活动 bigram（active bigram）

Active bigram：当前词表中仍然存在的符号对（可能被合并）。

Inactive bigram：已经被合并、不会再出现的 bigram。

通过 **优先队列（heap）**维护 bigram 出现次数。

每次只合并 频率最高的 active bigram → 不用每次全表扫描。

(2) 增量更新频率

合并某个 bigram 后，**只更新受影响的相邻 bigram 的计数**。

不必每次重新统计整个词表。

极大减少了重复计算量。

(3) 优先队列 + 哈希表

bigram 出现次数用哈希表维护。

优先队列快速获取出现次数最多的 bigram。

合并只影响局部词，所以更新复杂度低。

**BBPE：Byte-level BPE**

文本 → UTF-8 字节序列 → BPE 合并 → 子词编码

WordPiece

总体流程同BPE，但是挑选best-bigram的指标从频率变成了PMI（点互信息，Pointwise Mutual Information）：

$\mathrm{PMI}(a,b)=\frac{P(a,b)}{P(a)P(b)}$，其中a，b是临近的subword

分词方法：使用FMM进行vocabulary匹配